# 미리 만들어놨던 W2V 파일 읽어오기

In [1]:
# !pip install catboost
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# !pip install sweetviz
import sweetviz as sv
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import cm
import matplotlib.font_manager as fm
rc('font',family= 'malgun gothic')
rc('axes', unicode_minus = False)

import math
%matplotlib inline
import seaborn as sns
# !pip install -U klib
import klib
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    

from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import VotingRegressor

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.base import ClassifierMixin
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm, tqdm_notebook
import platform
from itertools import combinations
from scipy.stats.mstats import gmean


import datetime
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.decomposition import PCA

C:\Users\inuya\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# !pip install -U gensim

In [2]:
train = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding = 'cp949')
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding = 'cp949')  

In [44]:
data = pd.concat([train, test]).reset_index(drop=True)

In [137]:
%%writefile word2vec_list.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')

# str_nm ~ buyer_nm까지 하나의 문장으로 만들어보기.
cat_list = ['str_nm','goodcd','brd_nm','corner_nm','pc_nm','part_nm','team_nm','buyer_nm']
# train data
w2v_train = train.loc[:,cat_list]
w2v_train.goodcd = w2v_train.goodcd.astype(str)
w2v_train['list'] = w2v_train.apply(lambda x : list(x),axis = 1)
w2v_train['list'] = w2v_train.list.apply(lambda x: ''.join(map(str,x)))

# test data
w2v_test = test.loc[:,cat_list]
w2v_test.goodcd = w2v_test.goodcd.astype(str)
w2v_test['list'] = w2v_test.apply(lambda x: list(x),axis = 1)
w2v_test['list'] = w2v_test.list.apply(lambda x: ''.join(map(str,x)))

# train/test data에 추가
train['list'] = w2v_train.list
test['list'] = w2v_test.list

### Make corpus
p_level = 'list'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x) # unique 값들 array형태
    bs = np.array([]) # 빈 array 생성
    np.random.seed(seed) # seed 지정
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))  # 복원추출
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 100 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_list = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_list = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Overwriting word2vec_list.py


In [138]:
%run word2vec_list.py

### Feature Merge

#### 하나의 문장 list로 만든 word2vec 추가

In [139]:
w2v_features_train = pd.read_csv(os.path.abspath('../input') + '/seo_w2v_features_train.csv',encoding = 'utf-8')
w2v_features_test = pd.read_csv(os.path.abspath('../input') + '/seo_w2v_features_test.csv',encoding = 'utf-8')

In [140]:
del X_train_list['custid']
del X_test_list['custid']

X_train_list.columns = X_train_list.columns.map(lambda x : "list_" + str(x))
X_test_list.columns = X_test_list.columns.map(lambda x : "list_" + str(x))

w2v_features_train_new = pd.concat([w2v_features_train,X_train_list],axis = 1)
w2v_features_test_new = pd.concat([w2v_features_test,X_test_list],axis = 1)

In [141]:
w2v_features_train_new.to_csv('seo_w2v_features_train_new.csv', index=False,encoding = 'utf-8')
w2v_features_test_new.to_csv('seo_w2v_features_test_new.csv', index=False,encoding = 'utf-8')